### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from neuropy import plotting
import subjects
from neuropy.core import BinnedSpiketrain
import ev_utils

### During quiet waking only
- Reviewers had raised an important point that we are comparing waking/quiet-waking to sleep rather than sleep-deprivation to sleep.
- To address this we thought of provided explained variance for quiet waking period.

In [ ]:
from neuropy.core import BinnedSpiketrain
import pingouin as pg

sessions = subjects.pf_sess()

ev_df = []
for s, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 0.8  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    # zt_epochs = sess.paradigm.from_array(
    #     [post[0], post[0] + 5 * 3600],
    #     [post[0] + 5 * 3600, post[0] + 7.5 * 3600],
    #     ["0-5", "5-7.5"],
    # )
    zt_epochs = sess.get_zt_epochs(include_pre=False, include_maze=False)
    qw = sess.brainstates["QW"]

    pre_corr = get_corr(pre)
    maze_corr = get_corr(maze)

    ev, rev = [], []
    for e in zt_epochs.itertuples():
        if sess.tag == "NSD":
            e_qw = qw.time_slice(e.start, e.stop, strict=False)
        else:
            e_qw = sess.paradigm.from_array([e.start], [e.stop])

        e_spkcounts = np.hstack(neurons.get_spikes_in_epochs(e_qw, bin_size=0.25)[0])
        e_binspk = BinnedSpiketrain(
            e_spkcounts,
            0.25,
            neuron_ids=neurons.neuron_ids,
            shank_ids=neurons.shank_ids,
        )
        e_corr = e_binspk.get_pairwise_corr(pairs_bool)
        e_df = pd.DataFrame(dict(pre=pre_corr, maze=maze_corr, e=e_corr))
        kw = dict(data=e_df, x="maze")
        ev.append(pg.partial_corr(**kw, y="e", covar="pre").r.values[0] ** 2)
        rev.append(pg.partial_corr(**kw, y="pre", covar="e").r.values[0] ** 2)

    ev_diff = np.array(ev) - np.array(rev)
    ev_perc = (ev_diff / np.array(rev)) * 100

    ev_df.append(
        pd.DataFrame(
            {
                "ev": ev,
                "rev": rev,
                "ev_diff": ev_diff,
                "ev_perc": ev_perc,
                "zt": zt_epochs.labels,
                "session": s,
                "name": sess.animal.name + sess.animal.day,
                "sex": sess.animal.sex,
                "grp": sess.tag,
            }
        )
    )

ev_df = pd.concat(ev_df, ignore_index=True)
subjects.GroupData().save(ev_df, "ev_in_qw")

In [ ]:
from plotters import stripplot

fig = plotting.Fig(5, 4)

ax = fig.subplot(fig.gs[0])

# sns.boxplot(data=ev_df,x='zt',y='ev',hue='grp',showfliers=False)
stripplot(data=ev_df, x="zt", y="ev_diff", hue="grp", dodge=True, stat_anot=True, ax=ax)
ax.set_ylabel("Explained variance (above chance level)")
ax.set_xlabel("ZT bins")

### Bootstrap/Resample pairs comparision between SD and NSD

In [ ]:
import pingouin as pg
from ev_funcs import sd_delta_ev

sessions = subjects.pf_sess()
pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    zt_epochs = sess.get_zt_epochs()

    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 0.8  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    df = pd.DataFrame()
    # df["pre"] = get_corr(pre)
    # df["maze"] = get_corr(maze)

    for i, e in enumerate(zt_epochs.itertuples()):
        df[e.label] = get_corr([e.start, e.stop])

    df["session"] = sub
    df["grp"] = sess.tag

    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)
# subjects.GroupData().save(ev_df, "ev_chunks_shuffle")

# ------ Shuffling  -------
rng = np.random.default_rng()
ev_df = []
# ev_df.append(
#     pd.DataFrame(dict(delta_ev=get_ev(pcorr_df), zt=zt_epochs.labels[2:], method="exp"))
# )
data_ev = sd_delta_ev(pcorr_df)

pcorr_maze = (
    pcorr_df.loc[:, ["PRE", "MAZE", "session", "grp"]].copy().reset_index(drop=True)
)
pcorr_post = pcorr_df.loc[:, ["0-2.5", "2.5-5", "5-7.5"]].copy().reset_index(drop=True)

ev = []
for i in range(1000):
    shuff_df = pd.concat(
        [pcorr_maze, pcorr_post.sample(frac=1, replace=True).reset_index(drop=True)],
        axis=1,
    )
    shuff_df = pd.DataFrame(
        dict(
            delta_ev=data_ev - sd_delta_ev(shuff_df),
            zt=zt_epochs.labels[2:],
            method="shuffle",
        )
    )
    ev_df.append(shuff_df)

ev_df = pd.concat(ev_df, ignore_index=True)

In [ ]:
fig = plotting.Fig(5, 4)

ax = fig.subplot(fig.gs[0])
sns.boxplot(data=ev_df, x="zt", y="delta_ev", hue="method", ax=ax)
# fig.savefig(subjects.figpath_sd / "delta_ev_with_shuffle")
# ax.set_yscale('log')

### Compared pooled reactivation with shuffles in 15 minutes time windows
- In this I calculated EV using pooled pairwise correlations across sessions (while excluding one session each time) and comparing that to shuffled (across NSD and SD) pairs with replacement.

From Kamran
>Another good control (supplementary) would be to repeat it n times by excluding each of n animals. This would confirm that no single animal is biasing the results.
- Two ways of plotting it, one will difference in EV between NSD and SD sessions or plotting each curve separately with their shuffles.

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    zt_epochs = sess.get_sliding_zt_epochs(window=900, slideby=300)

    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 0.8  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    df = pd.DataFrame()

    for i, e in enumerate(zt_epochs.itertuples()):
        df[e.label] = get_corr([e.start, e.stop])

    df["session"] = sub
    df["grp"] = sess.tag

    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

In [ ]:
from ev_funcs import sd_delta_ev

ev_df = []
non_shuffled_col = ["PRE", "MAZE", "session", "grp"]

# ===== Leave one session out =========
n_sessions = len(sessions)
for i in tqdm(range(n_sessions)):
    df = pcorr_df[pcorr_df["session"] != i]  # one session left out

    # Uncomment if using difference of explained variance
    # ev_df.append(
    #     pd.DataFrame(
    #         dict(delta_ev=sd_delta_ev(df), zt=zt_epochs.labels[2:], method="exp")
    #     )
    # )
    _, nsd_ev, sd_ev = sd_delta_ev(df)

    ev_df.append(pd.DataFrame(dict(ev=nsd_ev, zt=zt_epochs.labels[2:], method="NSD")))
    ev_df.append(pd.DataFrame(dict(ev=sd_ev, zt=zt_epochs.labels[2:], method="SD")))

    # ====== Shuffling ===========
    df_maze = df.loc[:, non_shuffled_col].copy().reset_index(drop=True)
    df_post = df.loc[:, ~df.columns.isin(df_maze.columns)].copy().reset_index(drop=True)

    for i in range(100):
        df_shuff = pd.concat(
            [df_maze, df_post.sample(frac=1, replace=True).reset_index(drop=True)],
            axis=1,
        )
        # Uncomment if using difference of explained variance
        # df = pd.DataFrame(
        #     dict(
        #         delta_ev=sd_delta_ev(df_shuff),
        #         zt=zt_epochs.labels[2:],
        #         method="shuffle",
        #     )
        # )
        _, nsd_shuff, sd_shuff = sd_delta_ev(df_shuff)

        ev_df.append(
            pd.DataFrame(dict(ev=nsd_shuff, zt=zt_epochs.labels[2:], method="NSD_shuff"))
        )
        ev_df.append(
            pd.DataFrame(dict(ev=sd_shuff, zt=zt_epochs.labels[2:], method="SD_shuff"))
        )

ev_df = pd.concat(ev_df, ignore_index=True)

In [ ]:
# Percentage sleep in the background
nrem_df = []
for sess in sessions:
    zt_epochs = sess.get_sliding_zt_epochs()[2:]
    states = sess.brainstates["NREM"]
    nrem_dur = []
    for e in zt_epochs.itertuples():
        nrem_dur.append(states.time_slice(e.start, e.stop, strict=False).durations.sum())
    nrem_dur = np.array(nrem_dur)

    nrem_df.append(
        pd.DataFrame(dict(zt=zt_epochs.labels, nrem=nrem_dur / (15 * 60), grp=sess.tag))
    )

nrem_df = pd.concat(nrem_df, ignore_index=True)
nrem_df["zt"] = nrem_df["zt"].values.astype(float)

ev_df["zt"] = ev_df["zt"].values.astype(float)

fig = plotting.Fig(4, 2)
palette = subjects.colors_sd(1) + subjects.colors_sd(1.5)
ax = fig.subplot(fig.gs[0])
sns.lineplot(
    data=ev_df,
    x="zt",
    y="ev",
    hue="method",
    errorbar="sd",
    palette=palette,
    err_kws=dict(ec=None),
)
ax.set_ylabel("Explained variance")
ax.set_xlabel("Time(h)")
ax.set_title("Reactivation using pooled pairwise correlation")
fig.legend_with_text_only(ax)

ax2 = ax.twinx()

for g, grp in enumerate(["NSD", "SD"]):
    df = nrem_df[nrem_df.grp == grp]
    mean_nrem = df.groupby("zt").mean(numeric_only=True)["nrem"]
    ax2.fill_between(mean_nrem.index, 0, mean_nrem, step="mid", alpha=0.5)

fig.toggle_spines(ax2, sides=["right"], keep=True)
ax2.set_ylabel("NREM proportion")
ax2.legend("", frameon=False)
fig.savefig(subjects.figpath_sd / "pooled_ev_with_shuffle")

### QW for NSD and usual for SD sessions 

In [ ]:
from neuropy.core import BinnedSpiketrain

sessions = subjects.pf_sess()

pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    zt_epochs = sess.get_sliding_zt_epochs(window=900, slideby=300)

    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 0.8  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    df = pd.DataFrame()

    # for i, e in enumerate(zt_epochs.itertuples()):
    #     df[e.label] = get_corr([e.start, e.stop])

    qw = sess.brainstates["QW"]

    pre_corr = get_corr(pre)
    maze_corr = get_corr(maze)

    ev, rev = [], []
    for e in zt_epochs.itertuples():
        e_qw = qw.time_slice(e.start, e.stop, strict=False)
        # if sess.tag == "NSD":
        #     e_qw = qw.time_slice(e.start, e.stop, strict=False)
        # else:
        #     e_qw = sess.paradigm.from_array([e.start], [e.stop])

        if e_qw.durations.sum() > 1:
            e_spkcounts = np.hstack(neurons.get_spikes_in_epochs(e_qw, bin_size=0.25)[0])
            e_binspk = BinnedSpiketrain(
                e_spkcounts,
                0.25,
                neuron_ids=neurons.neuron_ids,
                shank_ids=neurons.shank_ids,
            )
            e_corr = e_binspk.get_pairwise_corr(pairs_bool)
        else:
            e_corr = np.nan * np.ones(len(pre_corr))

        df[e.label] = e_corr

    df["session"] = sub
    df["grp"] = sess.tag

    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

In [ ]:
from ev_funcs import sd_delta_ev

ev_df = []
non_shuffled_col = ["PRE", "MAZE", "session", "grp"]

# ===== Leave one session out =========
n_sessions = len(sessions)
for i in tqdm(range(n_sessions)):
    df = pcorr_df[pcorr_df["session"] != i]  # one session left out

    # Uncomment if using difference of explained variance
    # ev_df.append(
    #     pd.DataFrame(
    #         dict(delta_ev=sd_delta_ev(df), zt=zt_epochs.labels[2:], method="exp")
    #     )
    # )
    _, nsd_ev, sd_ev = sd_delta_ev(df)

    ev_df.append(pd.DataFrame(dict(ev=nsd_ev, zt=zt_epochs.labels[2:], method="NSD")))
    ev_df.append(pd.DataFrame(dict(ev=sd_ev, zt=zt_epochs.labels[2:], method="SD")))

    # ====== Shuffling ===========
    df_maze = df.loc[:, non_shuffled_col].copy().reset_index(drop=True)
    df_post = df.loc[:, ~df.columns.isin(df_maze.columns)].copy().reset_index(drop=True)

    # for i in range(100):

    #     df_shuff = pd.concat(
    #         [df_maze, df_post.sample(frac=1, replace=True).reset_index(drop=True)],
    #         axis=1,
    #     )
    #     # Uncomment if using difference of explained variance
    #     # df = pd.DataFrame(
    #     #     dict(
    #     #         delta_ev=sd_delta_ev(df_shuff),
    #     #         zt=zt_epochs.labels[2:],
    #     #         method="shuffle",
    #     #     )
    #     # )
    #     _, nsd_shuff, sd_shuff = sd_delta_ev(df_shuff)

    #     ev_df.append(
    #         pd.DataFrame(
    #             dict(ev=nsd_shuff, zt=zt_epochs.labels[2:], method="NSD_shuff")
    #         )
    #     )
    #     ev_df.append(
    #         pd.DataFrame(dict(ev=sd_shuff, zt=zt_epochs.labels[2:], method="SD_shuff"))
    #     )

ev_df = pd.concat(ev_df, ignore_index=True)

In [ ]:
# Percentage sleep in the background
nrem_df = []
for sess in sessions:
    zt_epochs = sess.get_sliding_zt_epochs()[2:]
    states = sess.brainstates["QW"]
    nrem_dur = []
    for e in zt_epochs.itertuples():
        nrem_dur.append(states.time_slice(e.start, e.stop, strict=False).durations.sum())
    nrem_dur = np.array(nrem_dur)

    nrem_df.append(
        pd.DataFrame(dict(zt=zt_epochs.labels, nrem=nrem_dur / (15 * 60), grp=sess.tag))
    )

nrem_df = pd.concat(nrem_df, ignore_index=True)
nrem_df["zt"] = nrem_df["zt"].values.astype(float)

ev_df["zt"] = ev_df["zt"].values.astype(float)

fig = plotting.Fig(4, 2)
palette = subjects.colors_sd(1) + subjects.colors_sd(1.5)
ax = fig.subplot(fig.gs[0])
sns.lineplot(
    data=ev_df,
    x="zt",
    y="ev",
    hue="method",
    errorbar="sd",
    palette=palette,
    err_kws=dict(ec=None),
)
ax.set_ylabel("Explained variance")
ax.set_xlabel("Time(h)")
ax.set_title("Reactivation using pooled pairwise correlation")
fig.legend_with_text_only(ax)

ax2 = ax.twinx()

for g, grp in enumerate(["NSD", "SD"]):
    df = nrem_df[nrem_df.grp == grp]
    mean_nrem = df.groupby("zt").mean(numeric_only=True)["nrem"]
    ax2.fill_between(mean_nrem.index, 0, mean_nrem, step="mid", alpha=0.5)

fig.toggle_spines(ax2, sides=["right"], keep=True)
ax2.set_ylim(0, 1)
ax2.set_ylabel("NREM proportion")
ax2.legend("", frameon=False)
fig.savefig(subjects.figpath_sd / "pooled_ev_with_shuffle")

### Comparing decay of pairwise correlation between PRE and SD

In [ ]:
sessions = subjects.pf_sess()

pre_df, sd_df = [], []
for sub, sess in enumerate(tqdm(sessions)):
    neurons = sess.neurons_stable.get_neuron_type("pyr")

    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    pre_epoch = sess.get_zt_epochs()["PRE"]

    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 0.8  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    df = pd.DataFrame()

    # for i, e in enumerate(zt_epochs.itertuples()):
    #     df[e.label] = get_corr([e.start, e.stop])

    qw = sess.brainstates["QW"]

    pre_corr = get_corr(pre)
    maze_corr = get_corr(maze)

    ev, rev = [], []
    for e in zt_epochs.itertuples():
        if sess.tag == "NSD":
            e_qw = qw.time_slice(e.start, e.stop, strict=False)
        else:
            e_qw = sess.paradigm.from_array([e.start], [e.stop])
        if e_qw.durations.sum() > 1:
            e_spkcounts = np.hstack(neurons.get_spikes_in_epochs(e_qw, bin_size=0.25)[0])
            e_binspk = BinnedSpiketrain(
                e_spkcounts,
                0.25,
                neuron_ids=neurons.neuron_ids,
                shank_ids=neurons.shank_ids,
            )
            e_corr = e_binspk.get_pairwise_corr(pairs_bool)
        else:
            e_corr = np.nan * np.ones(len(pre_corr))

        df[e.label] = e_corr

    df["session"] = sub
    df["grp"] = sess.tag

    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

### EV NREM vs QW Hiro, my data, Grosmark data combined
- Each dataset will have separate curves, so total 6 curves

In [ ]:
from neuropy.core import BinnedSpiketrain

sessions = (
    subjects.Nsd().pf_sess + subjects.NsdHiro().allsess + subjects.NsdGrosmark().allsess
)

# sessions = subjects.NsdGrosmark().ratBuddy_06272013

pcorr_df = []

for sub, sess in enumerate(tqdm(sessions)):
    post = sess.paradigm["post"].flatten()
    zt_epochs = sess.get_sliding_zt_epochs(window=900, slideby=300)
    pre = zt_epochs["PRE"].flatten()
    maze = zt_epochs["MAZE"].flatten()
    zt_epochs = zt_epochs[2:]  # removed PRE and MAZE

    if sess.tag == "NSD_HM":
        neurons = sess.neurons.get_neuron_type("pyr")
        # Across shnaks only
        shank_ids = neurons.shank_ids
        pairs_bool = shank_ids.reshape(-1, 1) - shank_ids.reshape(1, -1)
        dataset_tag = "HM"

    elif sess.tag == "NSD_GM":
        neurons = sess.neurons.get_neuron_type("pyr")
        # Across shnaks only
        shank_ids = neurons.shank_ids
        pairs_bool = shank_ids.reshape(-1, 1) - shank_ids.reshape(1, -1)
        dataset_tag = "GM"

    else:
        neurons = sess.neurons_stable.get_neuron_type("pyr")
        wave_similarity = neurons.get_waveform_similarity()
        pairs_bool = wave_similarity < 0.8  # only pairs which are least similar
        dataset_tag = "BG"

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    df = pd.DataFrame()
    df["PRE"] = get_corr(pre)
    df["MAZE"] = get_corr(maze)

    for state_name in ["NREM", "QW"]:
        states = sess.brainstates[state_name]

        for e in zt_epochs.itertuples():
            e_states = states.time_slice(e.start, e.stop, strict=False)
            # if sess.tag == "NSD":
            #     e_qw = qw.time_slice(e.start, e.stop, strict=False)
            # else:
            #     e_qw = sess.paradigm.from_array([e.start], [e.stop])

            if e_states.durations.sum() > 1:
                # In grosmark Buddy session sometimes there are 'zero' duration epoch, so taking care of that
                e_states = e_states.duration_slice(min_dur=0.5)
                e_spkcounts = np.hstack(
                    neurons.get_spikes_in_epochs(e_states, bin_size=0.25)[0]
                )
                e_binspk = BinnedSpiketrain(
                    e_spkcounts,
                    0.25,
                    neuron_ids=neurons.neuron_ids,
                    shank_ids=neurons.shank_ids,
                )
                e_corr = e_binspk.get_pairwise_corr(pairs_bool)
            else:
                e_corr = np.nan * np.ones(len(df))

            df[f"{state_name}_{e.label}"] = e_corr

    df["session"] = sub
    df["grp"] = sess.tag
    df["dataset"] = dataset_tag

    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

In [ ]:
from ev_funcs import sd_delta_ev
import pingouin as pg

ev_df = []

n_sessions = len(sessions)
for dataset in ["BG", "HM", "GM"]:
    df_dataset = pcorr_df[pcorr_df.dataset == dataset]
    session_id = np.unique(df_dataset.session.values)

    for i in tqdm(session_id):
        df = df_dataset[df_dataset["session"] != i]  # one session left out
        df = df.dropna(axis=1, how="all")

        columns = df.columns
        epochs = columns[~df.columns.isin(["PRE", "MAZE", "grp", "session", "dataset"])]

        try:
            ev = np.zeros(len(epochs))
            for e, epoch in enumerate(epochs):
                r = pg.partial_corr(data=df, x="MAZE", y=epoch, covar="PRE").r.values
                ev[e] = r**2
        except:
            ev = np.nan * np.zeros(len(epochs))

        new_labels = [f"{dataset}-{_}" for _ in epochs]
        ev_df.append(pd.DataFrame(dict(ev=ev, zt=new_labels)))
        # ev_df.append(pd.DataFrame(dict(ev=sd_ev, zt=zt_epochs.labels[2:], method="SD")))

ev_df = pd.concat(ev_df, ignore_index=True)
# separating zt names to float and adding a column for states
ev_df["state"] = [_.split("_")[0] for _ in ev_df["zt"]]  # separate dataset/state name
ev_df["zt"] = [float(_.split("_")[1]) for _ in ev_df["zt"]]  # string to float

In [ ]:
# from palettable.colorbrewer.qualitative import Paired_6_r,Dark2_6,Accent_6,Pastel2_6
from palettable.cartocolors.qualitative import Bold_6, Vivid_6, Pastel_6

# Percentage sleep in the background
nrem_df = []
for sess in sessions:
    zt_epochs = sess.get_sliding_zt_epochs()[2:]
    states = sess.brainstates["QW"]
    nrem_dur = []
    for e in zt_epochs.itertuples():
        nrem_dur.append(states.time_slice(e.start, e.stop, strict=False).durations.sum())
    nrem_dur = np.array(nrem_dur)

    nrem_df.append(
        pd.DataFrame(dict(zt=zt_epochs.labels, nrem=nrem_dur / (15 * 60), grp=sess.tag))
    )

nrem_df = pd.concat(nrem_df, ignore_index=True)
nrem_df["zt"] = nrem_df["zt"].values.astype(float)


fig = plotting.Fig(4, 2)
palette = subjects.colors_sd(1) + subjects.colors_sd(1.5)
ax = fig.subplot(fig.gs[0])
sns.lineplot(
    data=ev_df,
    x="zt",
    y="ev",
    hue="state",
    errorbar="se",
    palette=Vivid_6.mpl_colors,
    err_kws=dict(ec=None),
)
ax.set_ylabel("Explained variance")
ax.set_xlabel("Time(h)")
ax.set_title("Reactivation using pooled pairwise correlation (Hiro's data, NREM only)")
fig.legend_with_text_only(ax)

# ax2 = ax.twinx()

# for g, grp in enumerate(["NSD", "SD", "NSD_Hiro"]):
#     df = nrem_df[nrem_df.grp == grp]
#     mean_nrem = df.groupby("zt").mean(numeric_only=True)["nrem"]
#     ax2.fill_between(mean_nrem.index, 0, mean_nrem, step="mid", alpha=0.5)

# fig.toggle_spines(ax2, sides=["right"], keep=True)
# ax2.set_ylim(0, 1)
# ax2.set_xlim(0, 8)
# ax2.set_ylabel("")
# ax2.legend("", frameon=False)
fig.savefig(subjects.figpath_sd / "alldata_ev_nrem_qw")

### Hierarichal bootstraping of pairs within sessions with replacement
- Inspired from Saravanan et al., where they claim this way of bootstraping is better when the datasets are not independent. For example, in NSD and SD sessions were performed in the same animal.
- Can also be used for brainstate specific calculations 
- This generates two files: bootstrap_session_pairs_ev.npy (bootstraping at both session and pairs level) and bootstrap_session_only_ev.npy

In [ ]:
sessions = subjects.pf_sess()
pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    states = sess.brainstates
    zt_epochs = sess.get_zt_epochs()
    neurons = sess.neurons_stable.get_neuron_type("pyr")

    pcorr, labels = ev_utils.get_pcorr(
        neurons=neurons, epochs=zt_epochs, ignore_epochs=sess.artifact
    )
    df = pd.DataFrame(data=pcorr, columns=labels)

    # --- if statewise ------
    # pcorr1, labels1 = ev_utils.get_pcorr(
    #     neurons=neurons, epochs=zt_epochs[:2], ignore_epochs=sess.artifact
    # )
    # pcorr2, labels2 = ev_utils.get_pcorr(
    #     neurons=neurons, epochs=zt_epochs[2:], sub_epochs=states["QW"]
    # )
    # df = pd.DataFrame(
    #     data=np.hstack([pcorr1, pcorr2]), columns=np.concatenate([labels1, labels2])
    # )

    df["session"] = sub
    df["grp"] = sess.tag
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

In [ ]:
from stats_utils import bootstrap_resample
from ev_utils import get_ev, get_ev_mean

kw = dict(df=pcorr_df, n_iter=10000, n_jobs=10)

for level in ["session", "samples", "both"]:
    ev_df = bootstrap_resample(level=level, apply=get_ev, **kw)
    if level == "session":
        fname = "ev_bootstrap_session"
    if level == "samples":
        fname = "ev_bootstrap_pairs"
    if level == "both":
        fname = "ev_bootstrap_session_pairs"

    subjects.GroupData().save(ev_df, fname)

for level in ["session", "samples", "both"]:
    ev_df = bootstrap_resample(level=level, apply=get_ev_mean, **kw)
    if level == "session":
        fname = "ev_bootstrap_session_mean"
    if level == "samples":
        fname = "ev_bootstrap_pairs_mean"
    if level == "both":
        fname = "ev_bootstrap_session_pairs_mean"

    subjects.GroupData().save(ev_df, fname)

In [ ]:
from plotters import violinplot_sd
from stats_utils import get_bootstrap_prob

fig = plotting.Fig(5, 5)

ax = fig.subplot(fig.gs[0])

violinplot_sd(
    data=ev_df,
    x="zt",
    y="ev",
    hue="grp",
    ax=ax,
    # showfliers=False,
    # palette=subjects.colors_sd(1),
    # saturation=1,
    split=False,
    stat_across=get_bootstrap_prob,
    stat_within=get_bootstrap_prob,
)
ax.legend("", frameon=False)
ax.set_xlabel("")
ax.set_ylabel("Explained variance")

# fig.savefig(subjects.figpath_sd / "hierarichal_bootstrap_mean_ev_session")

### EV during REM sleep (2.5 hour blocks)
- 23/06/2023: No differences were observed between NSD and SD for EV during REM. The magnitude of EV were similar between both conditions.


In [ ]:
sessions = subjects.pf_sess()

ev_df = []
for s, sess in enumerate(tqdm(sessions)):
    pre_maze_epochs = sess.get_zt_epochs()[:2]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    rem = sess.brainstates["REM"]

    pcorr_pre_maze, pre_maze_labels = ev_utils.get_pcorr(neurons, pre_maze_epochs)
    pre_maze_df = pd.DataFrame(pcorr_pre_maze, columns=pre_maze_labels)

    if sess.tag == "NSD":
        zt_epochs = sess.get_zt_epochs()[2:]
    if sess.tag == "SD":
        zt_epochs = sess.get_zt_epochs()[4:]

    pcorr_zt, _ = ev_utils.get_pcorr(neurons, zt_epochs, sub_epochs=rem)
    zt_df = pd.DataFrame(data=pcorr_zt, columns=zt_epochs.labels)

    df = pd.concat([pre_maze_df, zt_df], axis=1)
    df["session"] = s
    df["grp"] = sess.tag

    sess_ev = ev_utils.get_ev(df)
    sess_ev["session"] = s
    sess_ev["grp"] = sess.tag
    ev_df.append(sess_ev)

ev_df = pd.concat(ev_df, ignore_index=True)

In [ ]:
_, ax = plt.subplots()

sns.stripplot(data=ev_df, x="zt", y="ev", hue="grp")

### Are EV during correlated with EV during recovery sleep

In [ ]:
data = subjects.GroupData().ev_in_chunks
sd_data = data[data.grp == "SD"]
sd1_ev = sd_data[sd_data.zt == "0-2.5"]
rs_ev = sd_data[sd_data.zt == "5-7.5"]

x = sd1_ev["ev"]
y = rs_ev["ev"]
linfit = stats.linregress(x, y)
r = linfit.rvalue
m = linfit.slope
c = linfit.intercept

fig = subjects.SdFig().fig2_supp()
ax = fig.subplot(fig.gs[0])
ax.scatter(x, y, s=10, c=subjects.colors_sd()[1])
ax.plot(x, x * m + c, color="k")
ax.set_ylabel("Explained variance (RS)")
ax.set_xlabel("Explained variance (SD1)")
ax.set_title(f"r = {np.round(r,2)}, pvalue = {np.round(linfit.pvalue,2)}")
ax.set_ylim(0.02, 0.06)

fig.savefig(subjects.figpath_sd / "ev_SD1_vs_RS", format="svg")

### pairwise correlations (explained variance) for NREM only 
- Have pairwise correlations only specific to NREM 

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for s, sess in enumerate(tqdm(sessions)):
    pre_maze_epochs = sess.get_zt_epochs()[:2]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    nrem = sess.brainstates["NREM"]

    pcorr_pre_maze, pre_maze_labels = ev_utils.get_pcorr(neurons, pre_maze_epochs)
    pre_maze_df = pd.DataFrame(pcorr_pre_maze, columns=pre_maze_labels)

    zt_starts = np.arange(post[0], post[0] + 7.5 * 3600, 300)
    zt_stops = zt_starts + 900
    zt_labels = np.arange(len(zt_starts)).astype("str")
    zt_epochs = sess.paradigm.from_array(zt_starts, zt_stops, zt_labels)

    pcorr_zt, _ = ev_utils.get_pcorr(neurons, zt_epochs, sub_epochs=nrem)
    zt_df = pd.DataFrame(data=pcorr_zt, columns=zt_labels.astype("int"))

    df = pd.concat([pre_maze_df, zt_df], axis=1)
    df["session"] = s
    df["grp"] = sess.tag
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

subjects.GroupData().save(pcorr_df, "pairwise_correlations_NREM")

### pairwise correlations (explained variance) for WAKE only 
- Have pairwise correlations only specific to WAKE 

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for s, sess in enumerate(tqdm(sessions)):
    pre_maze_epochs = sess.get_zt_epochs()[:2]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wake = sess.brainstates.label_slice(["AW", "QW"]).set_labels("WK").merge_neighbors()

    pcorr_pre_maze, pre_maze_labels = ev_utils.get_pcorr(neurons, pre_maze_epochs)
    pre_maze_df = pd.DataFrame(pcorr_pre_maze, columns=pre_maze_labels)

    zt_starts = np.arange(post[0], post[0] + 7.5 * 3600, 300)
    zt_stops = zt_starts + 900
    zt_labels = np.arange(len(zt_starts)).astype("str")
    zt_epochs = sess.paradigm.from_array(zt_starts, zt_stops, zt_labels)

    pcorr_zt, _ = ev_utils.get_pcorr(neurons, zt_epochs, sub_epochs=wake)
    zt_df = pd.DataFrame(data=pcorr_zt, columns=zt_labels.astype("int"))

    df = pd.concat([pre_maze_df, zt_df], axis=1)
    df["session"] = s
    df["grp"] = sess.tag
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

subjects.GroupData().save(pcorr_df, "pairwise_correlations_WAKE")

### Pairwise correlations (explained variance) by sleep onset in each session
As in the paper we are emphasising comparison between normal sleep and recovery sleep, it makes more sense to have the explained variance curves aligned by the start of NREM rich period. So each session will have different begining time for explained variance curves. In addition add bootstrapping.
In addition have separate curves for waking.

**NRK note**: from below, i appears that BG defined the beginning of this "NREM rich period" criteria as the first NREM epoch lasting at last 2 minutes (120 seconds).

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for s, sess in enumerate(tqdm(sessions)):
    pre_maze_epochs = sess.get_zt_epochs()[:2]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    nrem = sess.brainstates["NREM"]
    good_nrem = (
        sess.brainstates.label_slice(["NREM"])
        .time_slice(*post, strict=False)
        .duration_slice(min_dur=120)
    )
    nrem_onset = good_nrem[0].flatten()[0]

    pcorr_pre_maze, pre_maze_labels = ev_utils.get_pcorr(neurons, pre_maze_epochs)
    pre_maze_df = pd.DataFrame(pcorr_pre_maze, columns=pre_maze_labels)

    zt_starts = np.arange(nrem_onset, post[0] + 7.5 * 3600, 300)
    zt_stops = zt_starts + 900
    zt_labels = np.arange(len(zt_starts)).astype("str")
    zt_epochs = sess.paradigm.from_array(zt_starts, zt_stops, zt_labels)

    pcorr_zt, _ = ev_utils.get_pcorr(neurons, zt_epochs, sub_epochs=nrem)
    zt_df = pd.DataFrame(data=pcorr_zt, columns=zt_labels.astype("int"))

    df = pd.concat([pre_maze_df, zt_df], axis=1)
    df["session"] = s
    df["grp"] = sess.tag
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

subjects.GroupData().save(pcorr_df, "pairwise_correlations_aligned_by_NREM_onset")

### Generate pairwise correlations (explained variance) for WAKE
Similar to NREM ev, here collect pairwise correlations restricted to wake periods in POST for all sessions

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for s, sess in enumerate(tqdm(sessions)):
    pre_maze_epochs = sess.get_zt_epochs()[:2]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wake = sess.brainstates.label_slice(["AW", "QW"]).set_labels("WK").merge_neighbors()

    pcorr_pre_maze, pre_maze_labels = ev_utils.get_pcorr(neurons, pre_maze_epochs)
    pre_maze_df = pd.DataFrame(pcorr_pre_maze, columns=pre_maze_labels)

    zt_starts = np.arange(post[0], post[0] + 7.5 * 3600, 300)
    zt_stops = zt_starts + 900
    zt_labels = np.arange(len(zt_starts)).astype("str")
    zt_epochs = sess.paradigm.from_array(zt_starts, zt_stops, zt_labels)

    pcorr_zt, _ = ev_utils.get_pcorr(neurons, zt_epochs, sub_epochs=wake)
    zt_df = pd.DataFrame(data=pcorr_zt, columns=zt_labels.astype("int"))

    df = pd.concat([pre_maze_df, zt_df], axis=1)
    df["session"] = s
    df["grp"] = sess.tag
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)

subjects.GroupData().save(pcorr_df, "pairwise_correlations_aligned_by_WAKE")

### EDF 6C: Correlation between amount of QW and AW vs EV


In [ ]:
sessions = subjects.pf_sess()

wake_df = []
for s, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    aw = sess.brainstates.label_slice(["AW"])
    qw = sess.brainstates.label_slice(["QW"])

    zt_epochs = sess.get_zt_epochs(include_pre=False, include_maze=False)

    for e in zt_epochs.itertuples():
        aw_duration = aw.time_slice(e.start, e.stop, strict=False).durations.sum()
        qw_duration = qw.time_slice(e.start, e.stop, strict=False).durations.sum()
        # e_df = pd.DataFrame({"duration": e_duration}, index=[0])
        e_dur = e.stop - e.start
        e_df = pd.DataFrame(
            {"aw_dur": [aw_duration / e_dur], "qw_dur": [qw_duration / e_dur]}
        )
        e_df["zt"] = e.label
        e_df["session"] = s
        e_df["name"] = sess.animal.name + sess.animal.day
        e_df["grp"] = sess.tag
        wake_df.append(e_df)

wake_df = pd.concat(wake_df, ignore_index=True)
ev_df = subjects.GroupData().ev_in_chunks

ev_df = pd.merge(ev_df, wake_df, on=["zt", "session", "name", "grp"])

In [ ]:
fig = subjects.SdFig().fig_supp()

for i, zt in enumerate(["0-2.5", "2.5-5", "5-7.5"]):
    df = ev_df[(ev_df.grp == "SD") & (ev_df.zt == zt)]
    ax = fig.subplot(fig.gs[0, i])

    x = df["aw_dur"].values
    y = df["ev"].values
    linfit = stats.linregress(x, y)

    ax.scatter(x, y, s=2, c="r")
    ax.plot(x, linfit.slope * x + linfit.intercept, color="gray", lw=1, alpha=0.7)
    ax.set_title(f"r = {np.round(linfit.rvalue,2)}, p = {np.round(linfit.pvalue,3)}")

for i, zt in enumerate(["0-2.5", "2.5-5", "5-7.5"]):
    df = ev_df[(ev_df.grp == "SD") & (ev_df.zt == zt)]
    ax = fig.subplot(fig.gs[2, i])

    x = df["qw_dur"].values
    y = df["ev"].values
    linfit = stats.linregress(x, y)

    ax.scatter(x, y, s=2, c="r")
    ax.plot(x, linfit.slope * x + linfit.intercept, color="gray", lw=1, alpha=0.7)
    ax.set_title(f"r = {np.round(linfit.rvalue,2)}, p = {np.round(linfit.pvalue,3)}")

fig.savefig(subjects.figpath_sd / "qw_aw_EV_corr", format="svg")